# Covid 19 CT-Scan Image Segmentation

## Data Preprocessing

In [1]:
import glob
import pandas as pd
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

In [2]:
#Read and examine meta
raw_data = pd.read_csv('../input/covid19-ct-scans/metadata.csv')
raw_data.sample(5)

In [3]:
len(raw_data)

The files are nii format. so we use nibabel library to read them. </br>
Image are rotated. so we use <b>np.rot90</b> to account for that.

In [4]:
def read_nii(filepath):
    # Read nii file and return pixel array.
    ct_scan = nib.load(filepath)
    fdata = ct_scan.get_fdata() # type of categories
    print(fdata.shape)
    array = np.rot90(np.array(fdata)) # convert array and rotate
    return array

<code> Let's examine a file and it's annotations </code>

In [5]:
sample_ct = read_nii(raw_data.loc[0, 'ct_scan'])
sample_lung = read_nii(raw_data.loc[0, 'lung_mask'])
sample_infe = read_nii(raw_data.loc[0,'infection_mask'])
sample_all = read_nii(raw_data.loc[0, 'lung_and_infection_mask'])

<code>Note that <b>512x512</b> is the common resolution for ct images, also there are <b>301</b> slices in this ct scan.</code>

In [6]:
sample_ct.shape

Inspect the image

In [7]:
def plot_sample(array_list, color_map = 'nipy_spectral'):
    '''
    Plot a slice with all available annotations
    '''
    fig = plt.figure(figsize = (18,15))
    
    plt.subplot(1,4,1)
    plt.imshow(array_list[0], cmap='bone')
    plt.title('Original Image')

    plt.subplot(1,4,2)
    plt.imshow(array_list[0], cmap='bone')
    plt.imshow(array_list[1], alpha=0.5, cmap=color_map)
    plt.title('Lung Mask')

    plt.subplot(1,4,3)
    plt.imshow(array_list[0], cmap='bone')
    plt.imshow(array_list[2], alpha=0.5, cmap=color_map)
    plt.title('Infection Mask')

    plt.subplot(1,4,4)
    plt.imshow(array_list[0], cmap='bone')
    plt.imshow(array_list[3], alpha=0.5, cmap=color_map)
    plt.title('Lung and Infection Mask')

    plt.show()


There are 3 annotations available
* Lung Segmentation (contains different values to account for left and right lung)
* Infection Segmentation (segmentation of radiological findings that could be COVID)
* Lung + Infection (for the sake of making it simplier)

In [8]:
plot_sample([sample_ct[...,120], sample_lung[...,120], sample_infe[...,120], sample_all[...,120]])

In [9]:
def bulk_plot_sample(array_list, index_list):
    '''
    Plots multiple slices, wrapper of plot_sample
    '''
    for index_value in index_list:
        plot_sample([array_list[0][...,index_value], array_list[1][...,index_value],
                     array_list[2][...,index_value], array_list[3][...,index_value]])

In [10]:
bulk_plot_sample([sample_ct, sample_lung, sample_infe, sample_all], index_list=[100,110,120,130,140,150])

In [11]:
import cv2

In [12]:
print(sample_ct.shape[0])
print(sample_ct.shape[1])
print(sample_ct.shape[2])

In [13]:
lungs_list = []
infections_list = []

for i in range(len(raw_data)):
    ct = read_nii(raw_data['ct_scan'][i])
    infect = read_nii(raw_data['infection_mask'][i])
    
    for j in range(ct.shape[0]):
        lung_img = cv2.resize(ct[j], dsize= (128, 128), interpolation = cv2.INTER_AREA).astype('uint8')
        infec_img = cv2.resize(infect[j], dsize= (128, 128), interpolation = cv2.INTER_AREA).astype('uint8')
        
        lungs_list.append(lung_img[...,np.newaxis])
        infections_list.append(infec_img[...,np.newaxis])

In [14]:
lungs_list = np.array(lungs_list)
infections_list = np.array(infections_list)

In [15]:
lungs_list.shape

In [16]:
infections_list.shape

## Train Test Split

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
lung_train, lung_test, infect_train, infect_test = train_test_split(lungs_list, infections_list, test_size = 0.1)

In [19]:
lung_train.shape

## Building U-NET architecture

In [20]:
import tensorflow as tf #for deep learning

In [26]:
#Let's build the model

inputs = tf.keras.layers.Input((128, 128, 1))
normalized_input = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

# DOWN

c1 = tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same')(normalized_input)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)


c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)


c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)


c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
 
    
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

#UP

u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
 
u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
 
outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [27]:
history = model.fit(lung_train, infect_train, epochs = 10, validation_data = (lung_test, infect_test))

## Ploting the performance

In [28]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Val'], loc = 'upper left')
plt.show()

In [29]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train', 'Val'], loc = 'upper right')
plt.show()

## Let's predict

In [30]:
predicted = model.predict(lung_test)
fig = plt.figure(figsize = (18,15))

plt.subplot(1,3,1)
plt.imshow(lung_test[100][...,0], cmap = 'bone')
plt.title('original lung')

plt.subplot(1,3,2)
plt.imshow(lung_test[100][...,0], cmap = 'bone')
plt.imshow(infect_test[100][...,0],alpha = 0.5, cmap = "nipy_spectral")
plt.title('original infection mask')

plt.subplot(1,3,3)
plt.imshow(lung_test[100][...,0], cmap = 'bone')
plt.imshow(predicted[100][...,0],alpha = 0.5,cmap = "nipy_spectral")
plt.title('predicted infection mask')